In [10]:
import torch 
from torch.utils.data import DataLoader,random_split
import torchvision
from utils import (
    CustomImageNet100Dataset as ImageNetColorizationDataset,
    UNet32,
    rgb_to_lab,
    rgb_to_gray_with_clahe,
    lab_to_rgb_torch,
    train_model,
    evaluate_model,
    targeted_colorization_with_segmentation,
    sam_targeted_colorization
)
import os 
import subprocess

In [11]:
# Set device

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    print(f"Using device: {device}")

Using device: cuda


In [12]:

root_dir = "./data/imagenet100"

# Load train+val split from ImageNet
full_train_ds = ImageNetColorizationDataset(root_dir=root_dir, train=True, augment=True, img_size=224)
train_len = int(0.9 * len(full_train_ds))
val_len = len(full_train_ds) - train_len
train_ds, val_ds = random_split(full_train_ds, [train_len, val_len])

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=0, pin_memory=True)

# Load validation (test) set
test_ds = ImageNetColorizationDataset(root_dir=root_dir, train=False, augment=False, img_size=224)
test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=0, pin_memory=True)


In [13]:
# Initialize Model

model=UNet32(base_ch=64).to(device)
model.load_state_dict(torch.load("best_cifar_unet_clahe_lab.pth", map_location=device))

C:\Users\Amit Singh\AppData\Local\Temp\ipykernel_42792\2963291294.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_cifar_unet_clahe

<All keys matched successfully>

In [14]:
train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    save_path="best_imagenet_unet_clahe_lab.pth",
    epochs=75,
    patience=15
)

Using cuda


c:\Users\Amit Singh\Documents\Projects\nullClass_training-main\utils.py:82: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 6641 negative Z values that have been clipped to zero
  rgb_img = color.lab2rgb(lab_img.astype(np.float64))   # [H,W,3] in [0,1]
c:\Users\Amit Singh\Documents\Projects\nullClass_training-main\utils.py:82: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 1745 negative Z values that have been clipped to zero
  rgb_img = color.lab2rgb(lab_img.astype(np.float64))   # [H,W,3] in [0,1]
c:\Users\Amit Singh\Documents\Projects\nullClass_training-main\utils.py:82: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 297 negative Z values that have been clipped to zero
  rgb_img = color.lab2rgb(lab_img.astype(np.float64))   # [H,W,3] in [0,1]
c:\Users\Amit Singh\Documents\Projects\nullClass_training-main\utils.py:82: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted i

KeyboardInterrupt: 

In [ ]:
evaluate_model(model, val_loader, device)

NameError: name 'rgb_to_lab' is not defined